## 条件过滤

1. 读取本地数据
2. 基本面因子（过滤指数成分停牌涨跌停）
3. 技术因子与事件驱动（可买可卖）

## 1_读取本地数据

In [1]:
from jaqs.data import DataView
from jaqs.data import RemoteDataService
import os
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [2]:
dataview_folder = './JAQS_Data/hs300'
dv = DataView()
dv.load_dataview(dataview_folder)

Dataview loaded successfully.


## 3_过滤停牌涨跌停（可买可卖）

mask_index_member为要过滤的为True

can_enter与can_exit皆为可交易为True

In [3]:
from jaqs.util import dp
from jaqs.data.dataapi import DataApi

def mask_index():
    df_index_member = dv.get_ts('index_member')
    mask_index_member = df_index_member==0 #定义信号过滤条件-非指数成分
    return mask_index_member

def limit_up_down():
    # 定义可买卖条件——未停牌、未涨跌停
    trade_status = dv.get_ts('trade_status')
    mask_sus = trade_status == u'停牌'
    # 涨停
    dv.remove_field('up_limit')
    dv.add_formula('up_limit', '(close - Delay(close, 1)) / Delay(close, 1) > 0.095', is_quarterly=False, add_data=True)
    # 跌停
    dv.remove_field('down_limit')
    dv.add_formula('down_limit', '(close - Delay(close, 1)) / Delay(close, 1) < -0.095', is_quarterly=False, add_data=True)
    can_enter = np.logical_and(dv.get_ts('up_limit') < 1, ~mask_sus) # 未涨停未停牌
    can_exit = np.logical_and(dv.get_ts('down_limit') < 1, ~mask_sus) # 未跌停未停牌
    return can_enter,can_exit

In [4]:
mask_index_member = mask_index()
can_enter,can_exit = limit_up_down()

In [5]:
# dv.remove_field('mask_index_member')
# dv.remove_field('can_enter')
# dv.remove_field('can_exit')

In [6]:
dv.append_df(mask_index_member, 'mask_index_member')
dv.append_df(can_enter, 'can_enter')
dv.append_df(can_exit, 'can_exit')

Field [mask_index_member] is overwritten.
Field [can_enter] is overwritten.
Field [can_exit] is overwritten.


In [7]:
dv.save_dataview('./JAQS_Data/hs300')


Store data...
Dataview has been successfully saved to:
C:\Users\small\Desktop\PythonQTC\course\3_JAQS\JAQS_Data\hs300

You can load it with load_dataview('C:\Users\small\Desktop\PythonQTC\course\3_JAQS\JAQS_Data\hs300')


In [8]:
print(dv.get_ts('mask_index_member').head())

symbol     000001.SZ 000002.SZ 000008.SZ 000009.SZ 000027.SZ 000039.SZ  \
trade_date                                                               
20170103       False     False     False     False     False     False   
20170104       False     False     False     False     False     False   
20170105       False     False     False     False     False     False   
20170106       False     False     False     False     False     False   
20170109       False     False     False     False     False     False   

symbol     000046.SZ 000060.SZ 000061.SZ 000063.SZ    ...    601992.SH  \
trade_date                                            ...                
20170103        True     False     False     False    ...         True   
20170104        True     False     False     False    ...         True   
20170105        True     False     False     False    ...         True   
20170106        True     False     False     False    ...         True   
20170109        True     False     Fa